In [1]:
### INPUT IS THE DIRECTORY FOR ALL FILES
### OUTPUT: IF XML CONTAINS 3 LASER SACCADE THEN, OUTPUT = DF OD CLASSIFICATION OF EACH IND SACCADE - 
#% of all saccades in the test that are normal accuracy, hyper and hypo

In [1]:
#SETTING UP THE FILES TO BE PROCESSED


import os
import shutil

def move_files_to_main_directory(root_dir):
    """Moves all files from subfolders into the main directory."""
    for dirpath, _, filenames in os.walk(root_dir):
        for filename in filenames:
            if dirpath != root_dir:
                source_path = os.path.join(dirpath, filename)
                destination_path = os.path.join(root_dir, filename)
                shutil.move(source_path, destination_path)

root_directory = r"C:\Users\puvad\Downloads\april_1"
move_files_to_main_directory(root_directory)
print("Files moved successfully.")


In [3]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

def plot_saccade_waveform(folder_path, plot=False):
    results_df = pd.DataFrame()

    for filename in os.listdir(folder_path):
        if filename.endswith(".xml"):
            file_path = os.path.join(folder_path, filename)
            tree = ET.parse(file_path)
            root = tree.getroot()
            
            # Handle namespace dynamically
            namespace = {'ns': root.tag.split('}')[0].strip('{')} if '}' in root.tag else {}

            saccade_summary = []
            saccade_data = []

            # Extract both test and beat data in a single pass
            for saccade in root.findall('.//ns:VW_SaccadeTest', namespace):
                saccade_summary.append({
                    'TestUID': saccade.findtext('ns:TestUID', default='', namespaces=namespace),
                    'AvgAccuracyHRRightward': saccade.findtext('ns:AvgAccuracyHRRightward', default='0', namespaces=namespace),
                    'AvgAccuracyHRLeftward': saccade.findtext('ns:AvgAccuracyHRLeftward', default='0', namespaces=namespace),
                })

            for saccade in root.findall('.//ns:Saccade', namespace):
                saccade_data.append({
                    'TestUID': saccade.findtext('ns:TestUID', default='', namespaces=namespace),
                    'TimeMs': saccade.findtext('ns:TimeMs', default='0', namespaces=namespace),
                    'AccuracyPercent': saccade.findtext('ns:AccuracyPercent', default='0', namespaces=namespace),
                })
            
            if not saccade_summary or not saccade_data:
                continue  # Skip if data is missing

            summary_df = pd.DataFrame(saccade_summary).astype(float)
            saccade_df = pd.DataFrame(saccade_data).astype(float)

            merged_df = pd.merge(summary_df, saccade_df, on='TestUID', how='inner')

            # Compute statistics safely
            num_sac = max(len(saccade_data), 1)  # Prevent division by zero
            num_hyper = 100 * (saccade_df['AccuracyPercent'] > 120).sum() / num_sac
            num_hyper_low = 100 * (saccade_df['AccuracyPercent'] > 110).sum() / num_sac
            num_hypo = 100 * (saccade_df['AccuracyPercent'] < 70).sum() / num_sac
            num_nor = 100 - num_hyper - num_hypo

            results_summary = {
                "Filename": os.path.splitext(filename)[0],
                "Percent normal": num_nor,
                "Percent hyper": num_hyper,
                "Percent hyper_low": num_hyper_low,
                "Percent hypo": num_hypo
            }
            results_df = pd.concat([results_df, pd.DataFrame([results_summary])], ignore_index=True)

            if plot:
                plt.figure(figsize=(12, 6))
                plt.plot(merged_df['TimeMs'], merged_df['AccuracyPercent'], 'o-', label='Accuracy')
                plt.title(f'Saccade Waveform - {filename}')
                plt.xlabel('Time (ms)')
                plt.ylabel('Accuracy (%)')
                plt.legend()
                plt.show()

    return results_df

folder_path = r"C:\Users\puvad\Downloads\april_1"
data_summary = plot_saccade_waveform(folder_path)
data_summary.to_csv('2024_saccade_data_summary.csv')

KeyboardInterrupt: 